# This notebook is to build the db

In [146]:
#!/usr/bin/env python

import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
from datetime import datetime
from modules.myfuncs import *



## Get the data from portal using Socrata client

In [147]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:

url = "data.cityofchicago.org"
client = Socrata(url, None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

## Use a Socrata client query to get all data

In [148]:
# Red Light Cam data
rlc_cam = client.get("spqx-js37", #speed cams are at 'hhkd-xvj4' if you want to investigate?
                     where='violation_date BETWEEN \'2015-01-01T00:00:00.000\' AND \'2020-12-20T00:00:00.000\'',
                     limit=1000000,
                    )

client_df = pd.DataFrame.from_records(rlc_cam)  # Convert to pandas DataFrame





## Investigate results
We have all text objects.  

There are a fair number of missing locations/lat/long.  Hope to be able to replace those missing values.
This represents a large enough portion of dataset to look them up.

The na values for camera_id will have to be dropped

In [151]:
client_df.info()
client_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569594 entries, 0 to 569827
Data columns (total 10 columns):
intersection      569594 non-null object
camera_id         569594 non-null object
address           569594 non-null object
violation_date    569594 non-null object
violations        569594 non-null object
x_coordinate      539918 non-null object
y_coordinate      539918 non-null object
latitude          539918 non-null object
longitude         539918 non-null object
location          539918 non-null object
dtypes: object(10)
memory usage: 47.8+ MB


intersection          0
camera_id             0
address               0
violation_date        0
violations            0
x_coordinate      29676
y_coordinate      29676
latitude          29676
longitude         29676
location          29676
dtype: int64

In [152]:
#client_df.dropna(subset=['camera_id']).isna().sum()
client_df.dropna(subset=['camera_id'], inplace=True)
client_df.isna().sum()

intersection          0
camera_id             0
address               0
violation_date        0
violations            0
x_coordinate      29676
y_coordinate      29676
latitude          29676
longitude         29676
location          29676
dtype: int64

## Coerce my columns and make new ones

I forsee using month, day of week and year, so I will pull those out now.
I do not need location (I have lon and lat in other cols)
I do not neex xy coords.  I have lat/long

In [153]:
from datetime import datetime
client_df.info()
results_df = client_df.copy()

results_df['violations'] = results_df['violations'].apply(int)
results_df['latitude'] = results_df['latitude'].apply(float)
results_df['longitude'] = results_df['longitude'].apply(float)
results_df['violation_date'] = pd.to_datetime(results_df['violation_date'])
results_df['month'] = results_df['violation_date'].apply(lambda x: x.month)
results_df['weekday'] = results_df['violation_date'].apply(lambda x: datetime.weekday(x))
results_df['year'] = results_df['violation_date'].apply(lambda x: x.year)

results_df = results_df.drop(columns=['x_coordinate', 'y_coordinate', 'location'], index=1)
#results_df = results_df.drop(columns=['location'], index=1)

results_df.tail()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 569594 entries, 0 to 569827
Data columns (total 10 columns):
intersection      569594 non-null object
camera_id         569594 non-null object
address           569594 non-null object
violation_date    569594 non-null object
violations        569594 non-null object
x_coordinate      539918 non-null object
y_coordinate      539918 non-null object
latitude          539918 non-null object
longitude         539918 non-null object
location          539918 non-null object
dtypes: object(10)
memory usage: 47.8+ MB


,intersection,camera_id,address,violation_date,violations,latitude,longitude,month,weekday,year
569823,STONEY ISLAND AND 79TH,2464,1600 E 79TH ST,2020-12-20,10,41.751612,-87.585592,12,6,2020
569824,PULASKI AND NORTH,1992,1600 N PULASKI AVENUE,2020-12-20,1,41.909975,-87.726481,12,6,2020
569825,LAKE SHORE DR AND BELMONT,1413,400 W BELMONT AVE,2020-12-20,33,41.940241,-87.639639,12,6,2020
569826,ELSTON AND ADDISON,1403,3216 W ADDISON ST,2020-12-20,9,41.946735,-87.708610,12,6,2020
569827,CICERO AND I55,2252,4200 S CICERO AVENUE,2020-12-20,3,41.816729,-87.743537,12,6,2020


# Make a dataframe for individual cameras
We will try to add missing lat/long data using intersection addresses.
We will add start and end dates for our camera for those that went on/off line during that period of time.


# Create or connect to database

In [154]:
# Create a db
conn = create_connection('database/rlc.db')  # function from myfuncs file
c = conn.cursor()
#conn.close()


sqlite3 version: 2.6.0
connected to database/rlc.db


## Create a table for daily violations (all entries)

In [155]:

def create_table(c, mytable, cols):
    '''
    Creates a new table 
    #Sets a constraint on db (UNIQUE) so that if you have camera_id and violation_date the same...
    '''
    
    flat_cols = []
    for col in cols:
        for item in col:
            flat_cols.append(item) 
    
    my_sql = 'CREATE TABLE {} ' + '(' + '{} ' * len(flat_cols) + ');'
    my_sql = my_sql.format(mytable, *flat_cols)  # insert datatypes for cols

    print(my_sql) # just to see what I'm doing

    
    try:     
        c.execute(my_sql) 
    except Exception as e:
        print('\nCREATE TABLE', mytable, 'FAILED!!',  e)


        
cols = [
        ['intersection', 'text'], 
        ['camera_id', 'int'],
        ['address', 'text'],
        ['violation_date', 'numeric'],
        ['violations', 'int'],
        ['latitude', 'real'],
        ['longitude', 'real'],
        ['month', 'int'],
        ['week', 'int'],
        ['year', 'int'],
        ]

#print(*cols)
#sql = 'CREATE TABLE violations (intersection text, camera_id int, address text, violation_date numeric, violations int, latitude real, longitude real, month int, week int, year int)'
#c.execute(sql)

create_table(c, 'daily_violations', cols)
print()
conn.commit()

CREATE TABLE daily_violations (intersection text camera_id int address text violation_date numeric violations int latitude real longitude real month int week int year int );



In [156]:
## MOVED FUNCTIONS to myfuncs.py

In [157]:
sql_fetch_tables(c, conn)  # helper function in myfuncs
delete_all_entries(c, conn, 'daily_violations')

[('daily_violations',)]


## Now add some data

In [158]:
results_df.to_sql('daily_violations', conn, if_exists='replace', index = False)
#results_df.iloc[:10, :]

## Make sure it's there

In [159]:
# definitely have some INTEGER issue.  Had to change to text.  That's no good.
query = c.execute("SELECT camera_id, violations FROM daily_violations;").fetchall()
print(query[:5])
print(len(query))

[('2121', 19), ('1533', 2), ('1503', 2), ('2764', 3), ('1234', 1)]
569593


## Make a new table for individual cameras

In [160]:
# let's see if we can look at one camera and determine first and last date
cam_df = results_df.copy()
cam_df['start'] = cam_df['camera_id'].apply(lambda x: None)
cam_df['end'] = cam_df['camera_id'].apply(lambda x: None)

In [161]:

cam_df['violation_date'] = pd.to_datetime(cam_df['violation_date'])

cam_start = cam_df.groupby(['camera_id'])['violation_date'].min().reset_index()
cam_end = cam_df.groupby(['camera_id'])['violation_date'].max().reset_index()

cam_startend = cam_start.copy()
#print(cam_end[cam_end['camera_id']=='1503'].values[0][1])
cam_startend['end'] = cam_start['camera_id'].apply(lambda x: cam_end[cam_end['camera_id']==x].values[0][1])


# let's add the other columns we care about for cameras
#cam_startend['intersection'] = results_df.groupby(['camera_id'])


#print("START")
#results_df['start'] = results_df['camera_id'].apply(lambda x: cam_start[cam_start['camera_id']==x])['violation_date']

#print('END')
#results_df['end'] = results_df['camera_id'].apply(lambda x: cam_end[cam_end['camera_id']==x][:])

#results_df.head(100)

cam_startend.rename(columns={"violation_date": "start"}, inplace=True)

                                                    
cam_startend.isna().sum()
print(cam_startend.describe())
cam_startend.head()


       camera_id                start                  end
count        363                  363                  363
unique       363                   18                   18
top         1581  2015-01-01 00:00:00  2020-12-20 00:00:00
freq           1                  284                  256
first        NaN  2015-01-01 00:00:00  2015-03-02 00:00:00
last         NaN  2018-03-05 00:00:00  2020-12-20 00:00:00


,camera_id,start,end
0,1002,2015-01-01,2020-12-19
1,1003,2015-01-01,2020-12-19
2,1011,2015-01-02,2020-12-20
3,1014,2015-01-01,2020-12-20
4,1023,2015-01-02,2020-12-20


## Now make a table for my cam start and end data

In [162]:
cols = [
        ['camera_id', 'int'],
        ['start', 'numeric'],
        ['end', 'numeric'],
        ]

create_table(c, 'cam_startend', cols)

CREATE TABLE cam_startend (camera_id int start numeric end numeric );


In [163]:
sql_fetch_tables(c, conn)  # helper function in myfuncs
delete_all_entries(c, conn, 'cam_startend')

[('daily_violations',), ('cam_startend',)]


In [164]:
cam_startend.to_sql('cam_startend', conn, if_exists='replace', index = False)


In [165]:
query = c.execute("SELECT camera_id, start FROM cam_startend;").fetchall()
print(query[:5])
print(len(query))

[('1002', '2015-01-01 00:00:00'), ('1003', '2015-01-01 00:00:00'), ('1011', '2015-01-02 00:00:00'), ('1014', '2015-01-01 00:00:00'), ('1023', '2015-01-02 00:00:00')]
363


In [166]:
#cam_startend.T.apply(lambda x: x['camera_id'])

In [167]:
def get_minmax(df, cam_id, which):
    if which == 'start':
        return(df[df['camera_id']==cam_id]['start'].values[0])
    else:
        return(df[df['camera_id']==cam_id]['end'].values[0])

    
print(get_minmax(cam_startend, '1002', 'start'))

print(cam_startend.head())


2015-01-01T00:00:00.000000000
  camera_id      start        end
0      1002 2015-01-01 2020-12-19
1      1003 2015-01-01 2020-12-19
2      1011 2015-01-02 2020-12-20
3      1014 2015-01-01 2020-12-20
4      1023 2015-01-02 2020-12-20


In [168]:
#camloc_df = results_df.copy()
#camloc_df.groupby('camera_id').isna().sum().reset_index()
# this will be a dict for all of the intersections we look up.  We have fewer intersections than cams
#used_ints = {}  # for next part.  Only want to do this once.  There is a limit on geocode usage from geopy.
# DONT NEED THIS, JUST LOOK AT UNIQUES DUMMY

### We will try to make a db table that has camera locations and intersections

In [169]:
# Some of the addresses are truncated and not able to lookup with geocode

address_fix = {'2400 W VAN BUREN STREE': '2400 W VAN BUREN STREET',
               '4700 W IRVING PARK ROA': '4700 W IRVING PARK ROAD',
               '11500 S HALSTED STREE': '11500 S HALSTED STREET',
               '5500 S WENTWORTH AVEN': '5500 S WENTWORTH AVENUE',
                '10300 S HALSTED STREE': '10300 S HALSTED STREET',
               '3700 W IRVING PARK ROA': '3700 W IRVING PARK ROAD',
               '1600 W IRVING PARK ROA': '1600 W IRVING PARK ROAD',
               '7900 S JEFFERY BOULEV': '7900 S JEFFERY BOULEVARD',
               '2800 W IRVING PARK ROA': '2800 W IRVING PARK ROAD',
               '5200 W IRVING PARK ROA': '5200 W IRVING PARK ROAD',
               '3100 S DR MARTIN L KING': '3100 S MARTIN KING DRIVE',
               '1600 W DIVERSEY PARKWA': '1600 W DIVERSEY PARKWAY',
               '140 W KINZIE': '140 W Kinzie St',
                '150 N SACRAMENTO BOUL': '150 N SACRAMENTO BOUL',
               '800 N SACRAMENTO AVEN':'800 N SACRAMENTO AVENUE',
               '3200 N LAKESHORE DRIV':'3200 N LAKE SHORE DRIVE',
               '6400 W FULLERTON AVENU':'6400 W FULLERTON AVENUE',
               '6400 N MILWAUKEE AVEN':'6400 N MILWAUKEE AVENUE',
               '7900 S STONEY ISLAND':'7900 S Stony Island Ave',  
               '150 N SACRAMENTO BOUL':'150 N SACRAMENTO BOULEVARD',
                '3200 N LAKESHORE DRIVE':'3200 N Lake Shore Dr',
               '7900 S STONEY ISLAND AVENUE':'7900 S Stony Island Ave',
               '5600 W FULLERTON AVENU':'5600 W FULLERTON AVENUE',
               '8700 S LAFAYETTE AVEN':'8700 S LAFAYETTE AVENUE',
               '4400 N MILWAUKEE AVEN':'4400 N MILWAUKEE AVENUE',
              }


In [170]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https://github.com/sciencelee/chicago_rlc")

cam_locs = results_df.groupby(['camera_id', 'intersection']).max().reset_index()
cam_locs.head()

len(cam_locs)  # 364 total
len(cam_locs['camera_id'].unique()) # 363

cam_locs[cam_locs['camera_id'].duplicated()]  # 1421 is dupe
print(cam_locs[cam_locs['camera_id'] == '1421'])


print(results_df[(results_df['camera_id']=='1421') & (results_df['intersection']=='DAMEN AND DIVERSEY')].count())
print(results_df[(results_df['camera_id']=='1421') & (results_df['intersection']=='LARAMIE AND FULLERTON')].count())

# Turns out that a camera has two locations. One was only used one time.  We drop it.
cam_locs = cam_locs[(cam_locs['camera_id']!='1421') | (cam_locs['intersection']!='DAMEN AND DIVERSEY')]
print(len(cam_locs))  # 363 total (got rid of the bad one)


   camera_id           intersection                  address violation_date  \
83      1421     DAMEN AND DIVERSEY  2000 W DIVERSEY PARKWAY     2017-11-30   
84      1421  LARAMIE AND FULLERTON    2400 N LARAMIE AVENUE     2020-12-19   

    violations   latitude  longitude  month  weekday  year  
83           1  41.932394 -87.678173     11        3  2017  
84           6  41.924152 -87.756295     12        6  2020  
intersection      1
camera_id         1
address           1
violation_date    1
violations        1
latitude          1
longitude         1
month             1
weekday           1
year              1
dtype: int64
intersection      1210
camera_id         1210
address           1210
violation_date    1210
violations        1210
latitude          1210
longitude         1210
month             1210
weekday           1210
year              1210
dtype: int64
363


In [171]:
cam_locs.isna().sum()  # missing location for 19 cameras.  Let's fix it

camera_id          0
intersection       0
address            0
violation_date     0
violations         0
latitude          19
longitude         19
month              0
weekday            0
year               0
dtype: int64

In [172]:
'''
This section goes through all of the rlc and assigns latlong
Many lights are missing it.  
For each light, there is an address though.
We use geocoding to get the latlong
'''

# let's get all of the red light cameras with their gps location.  
# This will aid in placing the accidents at rlc intersections later (if closer than threshold point to point)
# Some RLCs are missing location data,  but have addresses.  I can use geocoding I guess to look them up.


#location = geolocator.geocode("175 5th Avenue NYC")
#print(location.address)
# out: Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...

#print((location.latitude, location.longitude))
# out: (40.7410861, -73.9896297241625)

#print(location.raw)
# out: {'place_id': '9167009604', 'type': 'attraction', ...}


# CAN USE THIS TO FIGURE OUT MY LAT LONG FROM RLC ADDRESS (or crash)   
 

def get_geocode(lat, long, address):
    if lat > 0:  # it's a location
        return (lat, long)
    else: # it's a proper location tuple, and assumed to be correct latlong
        if address in address_fix.keys(): address = address_fix[address]  # errors in the dataset chars omitted
        # if we make it this far, we have no record of this cam_id yet, and it doesn't have a proper location
        location = geolocator.geocode(address + ', Chicago, IL')
        if location == None:
            print(address+':'+address+' : could not geolocate') # print it out if we can't find (address errors)
        else:
            return (location.latitude, location.longitude)

        


#print(results_df.columns)
#print(results_df.loc[1,'address'])
#results_df['location'] = results_df['address'].apply(get_geocode)

# there is a clean one liner in python for this, I just know it.  Experiment here.
#results_df['latitude'], results_df['longitude'] = results_df.apply(lambda x: get_geocode(x.latitude, x.longitude, x.address), axis=1)
cam_locs['location'] = cam_locs.apply(lambda x: get_geocode(x.latitude, x.longitude, x.address), axis=1)



# some test code...
# for i in range(50):
#     x=results_df.iloc[i]
#     print('BEFORE', x.location, x.camera_id)
#     print('AFTER', get_geocode(x.location, 
#                       x.address, 
#                       x.camera_id, 
#                       x.intersection))
    


In [173]:
cam_locs.info()
cam_locs.isna().sum()  # missing location for 19 cameras.  Let's fix it

cam_locs['location'].head()
cam_locs['latitude'] = cam_locs['location'].apply(lambda x: x[0])
cam_locs['longitude'] = cam_locs['location'].apply(lambda x: x[1])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363 entries, 0 to 363
Data columns (total 11 columns):
camera_id         363 non-null object
intersection      363 non-null object
address           363 non-null object
violation_date    363 non-null datetime64[ns]
violations        363 non-null int64
latitude          344 non-null float64
longitude         344 non-null float64
month             363 non-null int64
weekday           363 non-null int64
year              363 non-null int64
location          363 non-null object
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 34.0+ KB


In [174]:
cam_locs.head()
cam_locs.isna().sum()

camera_id         0
intersection      0
address           0
violation_date    0
violations        0
latitude          0
longitude         0
month             0
weekday           0
year              0
location          0
dtype: int64

In [175]:
cam_locs.drop(columns=['violation_date', 'violations', 'month', 'weekday', 'year', 'location'], inplace=True)

In [176]:
        
cols = [
        ['camera_id', 'int'],
        ['intersection', 'text'], 
        ['address', 'text'],
        ['latitude', 'real'],
        ['longitude', 'real'],
        ]

create_table(c, 'cam_locations', cols)

CREATE TABLE cam_locations (camera_id int intersection text address text latitude real longitude real );


In [177]:

cam_locs.to_sql('cam_locations', conn, if_exists='replace', index = False)


In [178]:
query = c.execute("SELECT camera_id, latitude, longitude FROM cam_locations;").fetchall()
print(query[:5])
print(len(query))

[('1002', 41.85198384, -87.685785704), ('1003', 41.852141307, -87.685752642), ('1011', 41.990586042, -87.689822473), ('1014', 41.99060901, -87.689735481), ('1023', 41.953024751, -87.786683238)]
363


## Add a table of intersections with lat long
This should help us later determine if crash is at intersection

I chose to groupby the intersection and aggregate the most commonly occuring lat/long value

In [179]:
#results_df.groupby(['intersection', 'latitude', 'longitude']).reset_index()
intersection_df = results_df.groupby(['intersection']).agg({'latitude':pd.Series.mode,'longitude':pd.Series.mode,}).reset_index()


In [180]:
intersection_df
intersection_df.head()

,intersection,latitude,longitude
0,111TH AND HALSTED,41.6923,-87.6425
1,115TH AND HALSTED,41.6852,-87.6423
2,119TH AND HALSTED,41.6777,-87.6421
3,31ST AND CALIFORNIA,41.8373,-87.6952
4,31ST ST AND MARTIN LUTHER KING DRIVE,41.8385,-87.617


In [181]:
cols = [
        ['intersection', 'text'],
        ['latitude', 'real'], 
        ['longitude', 'real'],
        ]

create_table(c, 'intersection_locations', cols)

CREATE TABLE intersection_locations (intersection text latitude real longitude real );


In [182]:
sql_fetch_tables(c, conn)  # helper function in myfuncs
#delete_all_entries(c, conn, 'intersection_locations')

intersection_df.to_sql('intersection_locations', conn, if_exists='replace', index = False)


[('daily_violations',), ('cam_startend',), ('cam_locations',), ('intersection_locations',)]


In [183]:
query = c.execute("SELECT intersection FROM intersection_locations;").fetchall()
print(query[:5])
print(len(query))
intersection_df.head()


[('111TH AND HALSTED',), ('115TH AND HALSTED',), ('119TH AND HALSTED',), ('31ST AND CALIFORNIA',), ('31ST ST AND MARTIN LUTHER KING DRIVE',)]
183


,intersection,latitude,longitude
0,111TH AND HALSTED,41.6923,-87.6425
1,115TH AND HALSTED,41.6852,-87.6423
2,119TH AND HALSTED,41.6777,-87.6421
3,31ST AND CALIFORNIA,41.8373,-87.6952
4,31ST ST AND MARTIN LUTHER KING DRIVE,41.8385,-87.617


# Let's update our db once with new lat/long info


This was moved to preprocessing from EDA.

Should not have to do this again unless we reload the dataset.  May want to move this to bottom of the preprocessing section for organizational purposes.

In [193]:
# THIS TAKES SOME TIME (8min on macbook pro)

def read_loc(camloc_df, lat, long, cam_id):
    cam = camloc_df[camloc_df['camera_id']==cam_id]
    return (float(cam['latitude']), float(cam['longitude']))
        

read_loc(cam_locs, 45, 87, '1002')  # testing purposes
#results_df[:5].apply(lambda x: x.latitude, axis=1)  # tsting purpose

results_df['location'] = results_df.apply(lambda x: read_loc(cam_locs, x.latitude, x.longitude, x.camera_id), axis=1)



In [194]:
results_df['location']

0             (41.7802309, -87.6156396)
2             (41.9538663, -87.7166086)
3               (41.953838, -87.720427)
4             (41.9538663, -87.7166086)
5             (41.9538663, -87.7166086)
                      ...              
569823    (41.751611523, -87.585592284)
569824     (41.90997501, -87.726480523)
569825    (41.940241193, -87.639639114)
569826    (41.946734755, -87.708609774)
569827    (41.816729387, -87.743536712)
Name: location, Length: 569593, dtype: object

In [195]:
results_df['latitude'] = results_df['location'].apply(lambda x: x[0])
results_df['longitude'] = results_df['location'].apply(lambda x: x[1])



In [196]:
if 'location' in results_df.columns:
    results_df = results_df.drop(columns=['location'])

In [198]:
# write to the db
sql_fetch_tables(c, conn)  # helper function in myfuncs
delete_all_entries(c, conn, 'daily_violations')
results_df.to_sql('daily_violations', conn, if_exists='replace', index = False)


[('cam_startend',), ('cam_locations',), ('intersection_locations',), ('daily_violations',)]


Once run, we no longer need to revisit this notebook.  We have the dbs we wanted

In [ ]:
# also need to assign every rlc to a road segment
# we can use this https://stackoverflow.com/questions/24415806/coordinates-of-the-closest-points-of-two-geometries-in-shapely
# https://data.cityofchicago.org/Transportation/Chicago-Traffic-Tracker-Congestion-Estimates-by-Se/n4j6-wkkf
# This has segment ID numbers 1250 of them.
# Segments have start and end lon/lat, we can make into line.  
# rlcs can be points, and shapely has a distance method for finding dist between. Point.distance(Linestring)
# go through all of the Linestrings, then use the min to determine what segment it is on.  
# The segments are more than 300 miles total and are all major surface streets in Chicago
